In [5]:
import secrets
import hashlib
import ecdsa
from ecdsa import ellipticcurve
from ecdsa.curves import SECP256k1

#
# Python ZKP Example: Non-Interactive Schnorr Proof of Knowledge
#
# This script demonstrates a fundamental Zero-Knowledge Proof: proving knowledge
# of a discrete logarithm. In ECDSA terms, a "Prover" proves they know the
# private key `x` corresponding to a public key `P = x * G`, without revealing `x`.
#
# This is a non-interactive version using the Fiat-Shamir heuristic, which is
# a standard technique to build ZKPs for real-world applications.
#

# --- 1. Curve Parameters and Helper Functions ---
CURVE = SECP256k1
G = CURVE.generator
N = CURVE.order

def hash_points_and_message(*args):
    """
    Hashes a collection of points and messages into a single integer.
    The challenge in a non-interactive proof must be derived from public data.
    """
    hasher = hashlib.sha256()
    for arg in args:
        # Handle both Point and PointJacobi types from the ecdsa library
        if isinstance(arg, (ellipticcurve.Point, ellipticcurve.PointJacobi)):
            hasher.update(arg.x().to_bytes(32, 'big'))
            hasher.update(arg.y().to_bytes(32, 'big'))
        elif isinstance(arg, bytes):
            hasher.update(arg)
        elif isinstance(arg, str):
            hasher.update(arg.encode('utf-8'))
        else:
            raise TypeError(f"Unsupported type for hashing: {type(arg)}")

    return int.from_bytes(hasher.digest(), 'big') % N

# --- 2. The Prover's Logic ---
class Prover:
    """
    The Prover knows a secret and wants to prove their knowledge.
    """
    def __init__(self, private_key: int):
        if not (1 <= private_key < N):
            raise ValueError("Private key is out of range")
        self.private_key = private_key
        self.public_key = private_key * G

    def generate_proof(self, message: bytes = b''):
        """
        Generates the proof of knowledge. Follows the three steps:
        1. Commitment
        2. Challenge (derived via Fiat-Shamir)
        3. Response
        """
        # Step 1: Commitment
        # Generate a secret random nonce 'v' and compute the commitment point R = v * G
        nonce_v = secrets.randbelow(N)
        if nonce_v == 0:
            nonce_v = 1 # Ensure nonce is not zero
        R = nonce_v * G

        # Step 2: Challenge
        # In a non-interactive proof, the challenge 'e' is created by hashing
        # the public information: the prover's public key P, the commitment R,
        # and any optional message.
        e = hash_points_and_message(self.public_key, R, message)

        # Step 3: Response
        # Calculate s = v + e * x (mod N), where x is the private key.
        s = (nonce_v + e * self.private_key) % N

        # The final proof consists of the commitment point R and the response s
        return (R, s)


# --- 3. The Verifier's Logic ---
class Verifier:
    """
    The Verifier checks the Prover's proof without learning the secret.
    """
    def verify_proof(self, public_key: ellipticcurve.Point, proof: tuple, message: bytes = b''):
        """
        Verifies the proof. Returns True if valid, False otherwise.
        """
        R, s = proof

        # If R or s are invalid, fail.
        # Use direct comparison to INFINITY to handle both Point and PointJacobi types
        if R == ellipticcurve.INFINITY or not (1 <= s < N):
            return False

        # To verify, the verifier must re-calculate the same challenge 'e'
        # that the prover used.
        e = hash_points_and_message(public_key, R, message)

        # The core verification check: Does s*G == R + e*P ?
        # Where P is the prover's public key.
        lhs = s * G
        rhs = R + e * public_key

        # If the points match, the proof is valid! The prover must have known
        # the private key corresponding to the public key.
        return lhs == rhs


# --- 4. Main Simulation ---
if __name__ == '__main__':
    print("--- ZKP Schnorr Proof of Knowledge Simulation ---")

    # Setup: A user has a private key and a corresponding public key
    my_private_key = secrets.randbelow(N)
    prover = Prover(private_key=my_private_key)
    my_public_key = prover.public_key

    print(f"\nProver's secret private key (x): {hex(my_private_key)}")
    print(f"Prover's public key (P):\n  x: {hex(my_public_key.x())}\n  y: {hex(my_public_key.y())}")

    # The Prover wants to prove knowledge of their key while signing a message
    message_to_sign = b"I am the rightful owner of this key"
    print(f"\nMessage being 'signed' with ZKP: {message_to_sign.decode()}")

    # The Prover generates the proof
    zk_proof = prover.generate_proof(message_to_sign)
    print("\nProver generates a proof (R, s):")
    print(f"  R (commitment point):\n    x: {hex(zk_proof[0].x())}\n    y: {hex(zk_proof[0].y())}")
    print(f"  s (response value): {hex(zk_proof[1])}")

    # The Verifier receives the public key, the message, and the proof
    verifier = Verifier()

    # The Verifier checks the proof
    is_valid = verifier.verify_proof(my_public_key, zk_proof, message_to_sign)

    print(f"\n--- Verification ---")
    print(f"Verifier checks if s*G == R + e*P...")
    print(f"Result: Proof is {'VALID' if is_valid else 'INVALID'}")

    if is_valid:
        print("\nSUCCESS: The Verifier is convinced the Prover knows the private key.")
    else:
        print("\nFAILURE: The proof is incorrect.")

    # --- Negative Test Case ---
    print("\n--- Negative Test Case (Malicious Prover) ---")
    # A malicious prover who does NOT know the private key tries to generate a proof
    fake_prover = Prover(private_key=secrets.randbelow(N)) # They generate their own keys
    # But they try to generate a proof for someone else's public key
    fake_proof = fake_prover.generate_proof(message_to_sign)

    print("A malicious prover tries to use their own proof for someone else's public key.")
    is_fake_valid = verifier.verify_proof(my_public_key, fake_proof, message_to_sign)
    print(f"Result of fake proof: {'VALID' if is_fake_valid else 'INVALID'}")
    if not is_fake_valid:
        print("SUCCESS: The fake proof was correctly rejected by the Verifier.")
